https://datasystemslab.github.io/GeoSpark/tutorial/geospark-sql-python/

https://github.com/DataSystemsLab/GeoSpark/tree/master/python

https://datasystemslab.github.io/GeoSpark/tutorial/geospark-core-python/

https://medium.com/@karijdempsey/efficient-geospatial-analysis-with-spark-363ba50c5248


spark-submit --master yarn --deploy-mode cluster  --conf spark.yarn.appMasterEnv.SPARK_HOME=/share/apps/spark/spark-2.4.0-bin-hadoop2.6 --conf spark.yarn.submit.waitAppCompletion=false --conf spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.speculation=false --conf spark.executorEnv.LANG=en_US.UTF-8 --conf spark.yarn.appMasterEnv.LANG=en_US.UTF-8 --driver-cores 20 --driver-memory 55G --num-executors 40 --executor-cores 15 --executor-memory 55G ./covid/py/merge-census-blocks-pyspark.py

In [1]:
import os
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,desc,row_number,col,year,month,dayofmonth,dayofweek,to_timestamp,size,isnan,lit,date_format,from_json,broadcast
from pyspark.sql.types import MapType, StringType, IntegerType, StructType, StructField, FloatType, ArrayType, DoubleType

from geospark.register import GeoSparkRegistrator
from geospark.utils import GeoSparkKryoRegistrator, KryoSerializer
from geospark.register import upload_jars
from geospark.utils.adapter import Adapter

In [2]:
upload_jars()

True

In [3]:
try:
    spark
except NameError:
    print('Create Spark')
    spark=SparkSession.builder.appName("").config(
    "spark.serializer", KryoSerializer.getName).config(
    "spark.kryo.registrator", GeoSparkKryoRegistrator.getName).getOrCreate()

Create Spark


In [4]:
GeoSparkRegistrator.registerAll(spark)

True

In [5]:
source='cuebiq'
country='US'
start_date='2020-01-01'
end_date='2020-03-19'
states=['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']

if os.getenv('CLUSTER')=='PRINCE':
    path_to_data='/scratch/spf248/covid/data'
    directories=[x.strftime('%Y%m%d')+'00' for x in pd.date_range(start_date,end_date)][:1]
    file='part-00000-0428e20d-9019-4cbf-b5ce-bc9414007fec-c000.csv.gz'
    states=['NJ']
else:
    path_to_data='/user/spf248/covid/data'
    directories=[x.strftime('%Y%m%d')+'00' for x in pd.date_range(start_date,end_date)]
    file='*'

In [6]:
schema= StructType([
StructField("_c0", FloatType(), False),
StructField("_c1", StringType(), False),
StructField("_c2", FloatType(), False),
StructField("_c3", FloatType(), False),
StructField("_c4", FloatType(), False),
StructField("_c5", FloatType(), False),
StructField("_c6", FloatType(), False),
StructField("_c7", StringType(), False),
StructField("_c8", StringType(), False),])

In [7]:
def load_pings(directory):
    
    pings=spark.read.option(
    'compression', 'gzip').option(
    'header','false').option(
    "multiLine", "true").option(
    'escape','"').option(
    "encoding", "UTF-8").option(
    "delimiter", "\t").schema(schema).csv(
    os.path.join(
    path_to_data,
    source,
    country,
    directory,
    file))
    
    column_names=[
    'timestamp',
    'cuebiq_id',
    'device_type',
    'latitude',
    'longitude',
    'accuracy',
    'time_zone_offset',
    'classification_type',
    'transformation_type']
    pings=pings.toDF(*column_names)
    pings=pings.withColumn("time",to_timestamp(pings["timestamp"]+pings["time_zone_offset"]))
    pings.createOrReplaceTempView("pings")

    pings=spark.sql("""select time
    , cuebiq_id
    , latitude
    , longitude
    , accuracy
    , classification_type
    , ST_Point(cast(pings.longitude as Decimal(24,20))
    , cast(pings.latitude as Decimal(24,20))) as point
    from pings
    """)
    pings.createOrReplaceTempView("pings")
    
    return pings

In [8]:
def load_admin(state):
    
    admin=spark.read.option(
    "header", "true").csv(
    os.path.join(
    path_to_data,
    'shapefiles',
    country,
    'polygons',
    state+'.csv'))
    admin.createOrReplaceTempView("admin")

    admin=spark.sql("""select admin.GEOID10 as GEOID10
    , ST_GeomFromText(admin.geometry) as polygon
    from admin
    """)
    
    admin.createOrReplaceTempView("admin")
    
    return admin

In [9]:
for directory in directories:
    
    pings=load_pings(directory)
    
    for state in states:
        
        admin=load_admin(state)
        admin.cache()
        
#         admin_broadcast=broadcast(admin)
        
#         admin_rdd = Adapter.toSpatialRdd(admin)

        pings_geocoded=spark.sql(
        """
            SELECT p.time
            , p.cuebiq_id
            , p.latitude
            , p.longitude
            , p.accuracy
            , p.classification_type
            , s.GEOID10
            FROM pings AS p, admin AS s
            WHERE ST_Intersects(p.point, s.polygon)
        """
        )

        pings_geocoded.write.mode("overwrite").parquet(
        os.path.join(path_to_data,source,country,'geocoded',state,directory))

In [10]:
pings_geocoded.show()

+--------------------+--------------------+--------+---------+--------+-------------------+---------------+
|                time|           cuebiq_id|latitude|longitude|accuracy|classification_type|        GEOID10|
+--------------------+--------------------+--------+---------+--------+-------------------+---------------+
|2020-01-01 11:01:...|222a36f677448d751...|40.86519|-74.13686|    15.0|      PERSONAL_AREA|340311755003010|
|2020-01-01 11:21:...|222a36f677448d751...|40.86519|-74.13686|     8.0|      PERSONAL_AREA|340311755003010|
|2020-01-01 11:21:...|222a36f677448d751...|40.86519|-74.13686|     8.0|      PERSONAL_AREA|340311755003010|
|2020-01-01 07:24:...|222a36f677448d751...|40.86519|-74.13686|     5.0|      PERSONAL_AREA|340311755003010|
|2020-01-01 07:22:...|222a36f677448d751...|40.86519|-74.13686|    15.0|      PERSONAL_AREA|340311755003010|
|2020-01-01 10:32:...|222a36f677448d751...|40.86519|-74.13686|    15.0|      PERSONAL_AREA|340311755003010|
|2020-01-01 08:11:...|222a36